In [ ]:
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set_theme(style="darkgrid")
sns.set_palette("husl")

In [ ]:
# Load JSON and flatten all iterations into one DataFrame
with open("cavity_design_log.json", "r") as f:
    data = json.load(f)

rows = []
for key, cfg in data.items():
    for h in cfg["design_history"]:
        rows.append({
            "Q": h["result"]["Q"],
            "V": h["result"]["V"],
            "Q/V": h["result"]["qv_ratio"],
            "resonance_nm": h["result"].get("resonance_nm", 0),
            "period": round(h["params"]["period_nm"]),
            "rx": round(h["params"]["hole_rx_nm"]),
            "ry": round(h["params"]["hole_ry_nm"]),
            "taper": h["params"]["num_taper_holes"],
            "min_a%": h["params"]["min_a_percent"],
        })

df = pd.DataFrame(rows).sort_values("Q", ascending=False).reset_index(drop=True)
df.index = df.index + 1  # 1-based ranking
df.index.name = "#"

target_wl = 737  # nm

print(f"Total simulations: {len(df)}")
print(f"Target wavelength: {target_wl} nm\n")
df

In [ ]:
# -- Q and Q/V vs global rank --
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(df.index, df["Q"], color=sns.color_palette("viridis", len(df)), edgecolor="black", linewidth=0.3)
axes[0].set_xlabel("Rank (by Q)", fontsize=12)
axes[0].set_ylabel("Q Factor", fontsize=12)
axes[0].set_title("All Simulations Ranked by Q", fontsize=14, fontweight="bold")

axes[1].bar(df.index, df["Q/V"], color=sns.color_palette("magma", len(df)), edgecolor="black", linewidth=0.3)
axes[1].set_xlabel("Rank (by Q)", fontsize=12)
axes[1].set_ylabel("Q/V Ratio", fontsize=12)
axes[1].set_title("Q/V Ratio (same Q ranking)", fontsize=14, fontweight="bold")

plt.tight_layout()
plt.show()

In [ ]:
# -- Q vs hole_rx scatter (colored by period) --
fig, ax = plt.subplots(figsize=(10, 5))
sc = ax.scatter(df["rx"], df["Q"], c=df["period"], cmap="viridis",
                s=80, edgecolors="black", linewidths=0.5, alpha=0.85)
plt.colorbar(sc, ax=ax, label="Period (nm)")
best = df.iloc[0]
ax.scatter([best["rx"]], [best["Q"]], color="crimson", s=300, zorder=5,
           marker="*", edgecolors="black", linewidths=1, label=f"Best Q: {best['Q']:,.0f}")
ax.set_xlabel("Hole rx (nm)", fontsize=12)
ax.set_ylabel("Q Factor", fontsize=12)
ax.set_title("Q vs Hole rx (colored by Period)", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

# -- Q vs Period scatter (colored by rx) --
fig, ax = plt.subplots(figsize=(10, 5))
sc = ax.scatter(df["period"], df["Q"], c=df["rx"], cmap="plasma",
                s=80, edgecolors="black", linewidths=0.5, alpha=0.85)
plt.colorbar(sc, ax=ax, label="Hole rx (nm)")
ax.scatter([best["period"]], [best["Q"]], color="crimson", s=300, zorder=5,
           marker="*", edgecolors="black", linewidths=1, label=f"Best Q: {best['Q']:,.0f}")
ax.set_xlabel("Period (nm)", fontsize=12)
ax.set_ylabel("Q Factor", fontsize=12)
ax.set_title("Q vs Period (colored by Hole rx)", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# -- Resonance wavelength vs Q --
fig, ax = plt.subplots(figsize=(10, 5))
sc = ax.scatter(df["resonance_nm"], df["Q"], c=df["rx"], cmap="plasma",
                s=80, edgecolors="black", linewidths=0.5, alpha=0.85)
plt.colorbar(sc, ax=ax, label="Hole rx (nm)")
ax.axvline(x=target_wl, color="crimson", linestyle="--", linewidth=2, label=f"Target: {target_wl} nm")
ax.axvspan(target_wl - 5, target_wl + 5, alpha=0.12, color="green", label="+/-5 nm window")
ax.set_xlabel("Resonance (nm)", fontsize=12)
ax.set_ylabel("Q Factor", fontsize=12)
ax.set_title("Q vs Resonance Wavelength", fontsize=14, fontweight="bold")
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

# -- Correlation heatmap --
fig, ax = plt.subplots(figsize=(9, 7))
corr_cols = ["Q", "V", "Q/V", "period", "rx", "ry", "taper", "min_a%"]
sns.heatmap(df[corr_cols].corr(), annot=True, fmt=".2f", cmap="RdBu_r", center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, annot_kws={"size": 10})
ax.set_title("Parameter Correlation Matrix", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# -- Top 3 best designs --
print("TOP 3 DESIGNS BY Q")
print("=" * 80)
for i in range(min(3, len(df))):
    r = df.iloc[i]
    print(f"  #{i+1}  Q={r['Q']:>10,.0f}  V={r['V']:.3f}  Q/V={r['Q/V']:>10,.0f}  "
          f"res={r['resonance_nm']:.1f}nm  p={r['period']}  rx={r['rx']}  ry={r['ry']}  "
          f"taper={r['taper']}  min_a={r['min_a%']}%")
print("=" * 80)

# Detuning from target
for i in range(min(3, len(df))):
    r = df.iloc[i]
    delta = r["resonance_nm"] - target_wl
    print(f"  #{i+1} detuning from {target_wl}nm target: {delta:+.1f} nm")